In [1]:
import gym
import vectorincrement

In [2]:
gym.make('VectorIncrement-v0')

VectorIncrement(n=10, s=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.])

In [3]:
import gin
from vectorincrement.observation_encoder import KerasEncoder

In [4]:
gin.enter_interactive_mode()
gin.parse_config_file('config/cartpole_linear.gin')
gin.bind_parameter('KerasEncoder.inp_shape', (3,))

In [5]:
enc = KerasEncoder()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
enc.model.get_weights()

[array([[-0.0282245 ,  0.15164585,  0.05059561,  0.07477023,  0.06818791,
          0.06431356,  0.01936953, -0.03515893,  0.04630061,  0.02918101,
         -0.04981802, -0.06341111, -0.06402551,  0.07061589,  0.08477973,
          0.03893289,  0.02049178, -0.00852861,  0.04606097,  0.0008537 ,
          0.00416226,  0.044717  , -0.10613394,  0.0857382 , -0.01548095,
         -0.00263271,  0.01784088, -0.02989388, -0.01911334, -0.09506367,
         -0.02833785,  0.05934572, -0.07130744,  0.0348708 ,  0.00028443,
         -0.00976497,  0.02604678,  0.01940436, -0.05138169, -0.0665985 ,
         -0.03456772, -0.02949309, -0.02538395,  0.01621257,  0.02039056,
         -0.01106093, -0.02817813, -0.03893549,  0.03860227,  0.01194224,
          0.02303044, -0.01375255, -0.02872605,  0.02405525,  0.06358768,
         -0.02976374, -0.11037685,  0.03311437,  0.01029644,  0.04388334,
         -0.10373659,  0.02921132, -0.05826107,  0.03440207, -0.00739966,
          0.11871111, -0.02494959, -0.

In [10]:
enc.model.get_weights()

[array([[-0.0282245 ,  0.15164585,  0.05059561,  0.07477023,  0.06818791,
          0.06431356,  0.01936953, -0.03515893,  0.04630061,  0.02918101,
         -0.04981802, -0.06341111, -0.06402551,  0.07061589,  0.08477973,
          0.03893289,  0.02049178, -0.00852861,  0.04606097,  0.0008537 ,
          0.00416226,  0.044717  , -0.10613394,  0.0857382 , -0.01548095,
         -0.00263271,  0.01784088, -0.02989388, -0.01911334, -0.09506367,
         -0.02833785,  0.05934572, -0.07130744,  0.0348708 ,  0.00028443,
         -0.00976497,  0.02604678,  0.01940436, -0.05138169, -0.0665985 ,
         -0.03456772, -0.02949309, -0.02538395,  0.01621257,  0.02039056,
         -0.01106093, -0.02817813, -0.03893549,  0.03860227,  0.01194224,
          0.02303044, -0.01375255, -0.02872605,  0.02405525,  0.06358768,
         -0.02976374, -0.11037685,  0.03311437,  0.01029644,  0.04388334,
         -0.10373659,  0.02921132, -0.05826107,  0.03440207, -0.00739966,
          0.11871111, -0.02494959, -0.

In [12]:
from observation_encoder_sb import KerasEncoderVecWrapper